In [7]:
import json
from pprint import pprint
import re
from os import listdir, SEEK_END
import datetime
import random

In [2]:
list_re = [
    r"(\/sarcasm)",
    r"(\[[!?]\](?!\())",
    r"(:\^(?! ?[D)(\[\]]))",
    r"(<\/?sarcasm>)",
    r"(&lt;\/?sarcasm&gt)",
    r"(#sarcasm)",
    r"( \/s(?![a-zA-Z0-9]))"
          ]
sarc = re.compile(r"(\.~)|(\. ~)")
strike = re.compile(r"(?P<strike>~~.*~~)")
sarcasm_re = re.compile('|'.join(list_re))

filename_re = re.compile(r"""(RC_20\d{2}-\d{2}(?!\.bz2))""")

In [3]:
def commentGenerator():
    for name in listdir():
        if filename_re.match(name) is not None:
            start = datetime.datetime.now()
            print("Processing file %s, starttime: %s"%(name, start))
            pop = []
            for n, line in enumerate(open(name)):
                try:
                    body = json.loads(line)['body']
                    if 10 <= len(body) <= 300:
                        compareText = body.lower()
                        if(sarcasm_re.search(compareText) is not None) or \
                        ((sarc.search(compareText) is not None) and (strike.search(compareText) is None)):
                            yield (True, body)
                            yield (False, random.choice(pop))
                            pop = []
                        else:
                            pop.append(body)
                            if len(pop) == 1000:
                                yield (False, random.choice(pop))
                                pop = []
                except:
                    print("Error on line %s, : %s"%(n, line))
            print("Finished: " + str(datetime.datetime.now()-start))

In [4]:
sarcasticCount = 0
seriousCount = 0
sarcastic = open('json/sarcastic0.json', 'a')
serious = open('json/serious0.json', 'a')
sarcastic.write('[\n')
serious.write('[\n')
for n, (sarcasm, comment) in enumerate(commentGenerator()):
    try:
        if sarcasm:
            json.dump({'text':comment}, sarcastic)
            sarcasticCount += 1
            if sarcasticCount % 100000 == 0:
                serious.write('\n]')
                sarcastic.close()
                sarcastic = open('json/sarcastic%d.json'%sarcasticCount, 'a')
            else:
                sarcastic.write(',\n')
        else:
            json.dump({'text':comment}, serious)
            seriousCount += 1
            if seriousCount % 100000 == 0:
                serious.write('\n]')
                serious.close()
                serious = open('json/serious%d.json'%seriousCount, 'a')
            else:
                serious.write(',\n')
    except:
        print("Error on index %s, : (%s, %s)"%(n, sarcasm, comment))
        sarcastic.close()
        serious.close()
        
sarcastic.close()
serious.close()

sarcasticname = sarcastic.name
seriousname = serious.name

sarcastic.close()
serious.close()

sarcastic = open(sarcasticname, 'r+b')
serious = open(seriousname, 'r+b')
sarcastic.seek(-3, SEEK_END)
sarcastic.write(bytes("\r\n]", 'utf8'))
serious.seek(-3, SEEK_END)
serious.write(bytes("\r\n]", 'utf8'))
sarcastic.close()
serious.close()

Processing file RC_2007-10, starttime: 2017-04-18 20:43:36.564866
Finished: 0:00:04.383147
Processing file RC_2007-11, starttime: 2017-04-18 20:43:40.948013
Finished: 0:00:10.006578
Processing file RC_2007-12, starttime: 2017-04-18 20:43:50.954591
Finished: 0:00:10.377783
Processing file RC_2008-01, starttime: 2017-04-18 20:44:01.332374
Finished: 0:00:13.393392
Processing file RC_2008-02, starttime: 2017-04-18 20:44:14.725766
Finished: 0:00:12.478454
Processing file RC_2008-03, starttime: 2017-04-18 20:44:27.204220
Finished: 0:00:12.737296
Processing file RC_2008-04, starttime: 2017-04-18 20:44:39.941516
Finished: 0:00:12.931537
Processing file RC_2008-05, starttime: 2017-04-18 20:44:52.873053
Finished: 0:00:14.713887
Processing file RC_2008-06, starttime: 2017-04-18 20:45:07.586940
Finished: 0:00:15.804731
Processing file RC_2008-07, starttime: 2017-04-18 20:45:23.391671
Finished: 0:00:16.186937
Processing file RC_2008-08, starttime: 2017-04-18 20:45:39.579607
Finished: 0:00:16.393982

Finished: 0:22:28.138716
Processing file RC_2015-03, starttime: 2017-04-19 08:10:13.515278
Finished: 0:26:55.485840
Processing file RC_2015-04, starttime: 2017-04-19 08:37:09.001118
Finished: 0:28:50.431441
Processing file RC_2015-05, starttime: 2017-04-19 09:05:59.432559
Finished: 0:27:39.150709


In [5]:
sarcastic.close()
serious.close()